# Setup

This is following [10 minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html) for pandas 1.4.3

In [ ]:
import numpy as np
import pandas as pd

# Object creation

In [ ]:
# Series - pass a list to pd.Series
s1 = pd.Series([1, 2, 3, np.nan, 5])

In [ ]:
s1

In [ ]:
# create an index to use in the dataframe
# default frequency is D (day), so creates a DatetimeIndex of 10 days
dates = pd.date_range('20220619', periods=10)
dates

just in case you are wondering, here's [all the "offset aliases"](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases) for Datetime

In [ ]:
# Dataframe - pass a Numpy array
# create a 10 row, 4 col random number array, index by dates, give some column names
df = pd.DataFrame(np.random.randn(10, 4), index=dates, columns=list('ABCD'))
df

In [ ]:
# Create a dataframe by passing a dictionary of objects
# where each object can be converted into a series-like structure
df2 = pd.DataFrame(
    {
        "A": 1.0,
        "B": pd.Timestamp("20220619"),
        "C": pd.Series(1, index=list(range(6)), dtype="float32"),
        "D": np.array([3]*6, dtype="int32"),
        "E": pd.Categorical(["test", "train", "test", "train", "test", "train"]),
        "F": list("foofoo"),
        "G": "foo"
    }
)

df2

In [ ]:
# the datatype of each of the columns would be different
df2.dtypes

# Viewing Data

In [ ]:
# top of the dataframe
df.head()

In [ ]:
# bottom 3 records of the dataframe
df.tail(3)

In [ ]:
# index of the df
df.index

In [ ]:
# columns of the df
df.columns

In [ ]:
# .to_numpy gives a NumPy representation of the dataframe
# this is expensive if all columns are of different data type
df.to_numpy()

In [ ]:
df2.to_numpy()

In [ ]:
# quick summary stats
df.describe()

1. **count** = Count number of non-NA/null observations
1. **max** = Maximum of the values in the object
1. **min** = Minimum of the values in the object
1. **mean** = Mean of the values
1. **std** = Standard deviation of the observations
1. **25%** = Default lower percentile
1. **50%** = 50 percentile - same as the median 
1. **75%** = Default upper percentile

In [ ]:
# change the percentiles
df.describe(percentiles=[.1, .5, .9])

In [ ]:
# E, F, G in d2 are not numeric, so do not come up in describe()
df2.describe()

In [ ]:
# transpose the data
# turn rows to columns and vice versa
df.T

### [Axis in pandas](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_axis.html?highlight=set_ax#pandas.DataFrame.set_axis)

```axis{0 or ‘index’, 1 or ‘columns’}```

Always remember that when you specify:
* ```axis=0``` or ```axis='index'``` you indicate that the operation should be along the _index_ (aka across the rows). More often than not indexes will go from top to bottom (vertically), but sometimes they may not (for e.g. if you use columns as index or when you have hierarchical or multi-index data).
* ```axis=1``` or ```axis='columns'``` indicates the operation is along the columns

When in doubt use the explicit version ```'index'``` or ```'columns'``` instead of ```0``` or ```1```

In [ ]:
# sort along the axis - 1 = horizontal
df.sort_index(axis=1, ascending=False)

In [ ]:
# sort along the axis - 0 = vertical
df.sort_index(axis=0, ascending=False)

In [ ]:
# sort ascending by values in a column
df.sort_values(by="A")

In [ ]:
# sort by non-numerical values
df2.sort_values(by="F", ascending=False)

In [ ]:
# sort by two or more columns
df2.sort_values(by=["F", "E"])

# Selection

For production prefer the following instead of other data access methods (typical python methods like ["col"] or [a:b] slices etc.):

```.at, .iat, .loc and .iloc.```

## Getting

In [ ]:
# selecting a single column returns a Series object
df["A"]

In [ ]:
# selecting a slice
df[1:5]

## Selection by label

In [ ]:
# selecting based on a label
df.loc[dates[0]]

In [ ]:
# select on a multi-axis by lable
# I honestly do not know what the comment above means...help!
df.loc[:, ["A", "B"]]

In [ ]:
# specific index value results in reduction of dimensions
res = df.loc["2022-06-20"]
print(res)
print("res.shape = ", res.shape, " vs. df.shape = ", df.shape)

In [ ]:
# get to a specific scalar:
#
# method one
df.loc[dates[0], "A"]

In [ ]:
#
# method two (slightly faster than method one)
df.at[dates[0], "A"]

## Selection by position

In [ ]:
df.iloc[2]

In [ ]:
# slices - similar to NumPy / Python - [row:slice, col:slice]
df.iloc[1:5, 0:2]

In [ ]:
# by list of locations - similar to NumPy / Python - [[list of rows], [list of cols]]
df.iloc[[0, 1, 2, 6], [0, 2]]

In [ ]:
# by list of locations - similar to NumPy / Python - [[list of rows], [list of cols]]
# change the order of columns, repeact a column
df.iloc[[0, 1, 2, 6], [2, 1, 0, 2]]

In [ ]:
# slice rows explicitly, keep all columns
df.iloc[[1, 2], :]

In [ ]:
# slice columns, keep all rows
df.iloc[:, [2, 3]]

In [ ]:
# everything, because you can
df.iloc[:, :]

In [ ]:
# get to a scalar (2 methods, just like before)
#
# method one: use iloc
df.iloc[1, 2]

In [ ]:
#
# method two: use iat
df.iat[1, 2]

## Boolean Indexing

In [ ]:
# use a value found in a single col to get data
df[df["A"] > 0]

In [ ]:
# boolean across the entire DF - vals that don't match go NaN
df[df > 0]

In [ ]:
# add another column
df11 = df.copy()
df11["E"] = ["one", "two", "three", "four",
             "two", "five", "one", "two", "three", "four"]
df11

In [ ]:
# the isin() query - basically the in clause
df11[df11["E"].isin(["two", "five"])]

## Setting values

In [ ]:
# matching indexes auto-aligns values
s1 = pd.Series(range(11, 21), index=pd.date_range('20220619', periods=10))
s1

In [ ]:
df["F"] = s1
df

In [ ]:
# setting values by label and position
# first let's make a quick copy
df12 = df.copy()
df12

In [ ]:
# set by label
df12.at[dates[0], "A"] = 0
# set by position
df12.iat[0, 1] = 0
df12

In [ ]:
# kinda bigger replacement
df12.loc[:, "D"] = np.array([5]*len(df))
df12

In [ ]:
# setting values using a boolean selection (aka where clause)
df12[df12 > 0] = -df12
df12

# Missing Data

### _reindex_
change/add/delete index on a specified axis, returns a new dataframe

In [ ]:
df13 = df.reindex(index=dates[0:4], columns=list(df.columns)+["G"])
df13

### handling missing data
1. _```np.nan```_
1. _```pandas.isna()```_
1. ```df.dropna()```
1. ```df.fillna()```

In [ ]:
# missing data in pandas is np.nan
df13.iat[1, 0] = np.nan
df13.iloc[1:, 5] = np.random.randint(1)
df13

In [ ]:
# get a boolean mask where values are NaN
df131 = pd.isna(df13)
df131

In [ ]:
# or just
pd.isna(df13)

In [ ]:
# the original is still there
df13

In [ ]:
# we are going to drop / replace values now, let's make a couple of copies of the dataframe
df132 = df13.copy()
df132

In [ ]:
# drop rows/columns that have missing data
# by default it returns a new dataframe, you may want to specify inplace=True for modifying current dataframe:
df_no_na = df132.dropna(how="any")
# how=‘any’ : If any NA values are present, drop that row or column.
# how=‘all’ : If all values are NA, drop that row or column.

In [ ]:
# all rows/cols with missing data stripped
df_no_na

In [ ]:
# original still intact
df132

In [ ]:
# drop missing data from original
df132.dropna(how="any", inplace=True)
df132

In [ ]:
# fill missing data
df133 = df13.copy()
df133.fillna(np.pi*1000)

# Align and Join

There needs to be a bigger notebook for this topic.
You need to know there's "joins" in Pandas just like in the SQL world, 
like join and left join and right join and inner and outer and all that...

## Create two datasets with _mismatched_ indexes

In [ ]:
# date range indexes
idx1 = pd.date_range('2022-01-01',periods = 10)
# 2022-01-01', '2022-01-02' don't exist in idx2
# '2022-01-11', '2022-01-12' don't exist in idx1
idx2 = pd.date_range('2022-01-03',periods = 10)

In [ ]:
# dataframes from indexes
d1 = pd.DataFrame(index = idx1, data = 
    {
        'A': np.random.rand(10),
        'B': np.random.randint(1, high=25, size = 10)
    }
)

d2 = pd.DataFrame(index = idx2, data = 
    {
        'A': np.random.rand(10),
        'B': np.random.randint(1, high=25, size = 10)
    }
)

## Quick Aside: Rendering two dataframes side-by-side

The trick of rendering 2 dataframes side-by-side was from [this](https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side) stackoverflow question.

In [ ]:
# we gon need to display both data frames side by side, so...
from IPython.display import display_html 
def render_df_side_by_side(a,b):
    a_styler = a.style.set_table_attributes("style='display:inline'")
    b_styler = b.style.set_table_attributes("style='display:inline'")
    display_html(a_styler._repr_html_()+b_styler._repr_html_(), raw=True)

In [ ]:
render_df_side_by_side(d1,d2)

Table rendering/styling options is [a bigger discussion](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html#) to be tackled later.

## .add, .sub, .mul, .div etc.

flexible wrappers (```add, sub, mul, div, mod, pow```) to arithmetic operators: ```+, -, *, /, //, %, **```

In [ ]:
# the +, -, *, /, //, %, ** operations align indexes to create a UNION of indexes
d3 = d1+d2
d4 = d1.sub(d2) #equivalent to d1-d2
d5 = d1*d2
d6 = d1.div(d2)

In [ ]:
# indexs not present in the other dataframe get a NaN
d3.style.highlight_null(null_color='red')

In [ ]:
# sub
d4.style.highlight_null(null_color='red')

In [ ]:
# mul
d5.style.highlight_null(null_color='red')

In [ ]:
# div
d6.style.highlight_null(null_color='red')

### removing the NaN values from the resultant dataframe

In [ ]:
# the + operation is UNION of indexes
d31 = d1+d2
# indexs not present in the other dataframe get a NaN
# do you can obvs remove nan values like before
d3.dropna().style.highlight_null(null_color='red')

## df.align()

Pandas [align](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.align.html) works to join the dataframes in interesting ways.

In [ ]:
# align returns two dataframes - left and right as a result of a join
# default join is outer - the union of indices will be used as index for left and right
left, right = d1.align(d2, join='outer')
render_df_side_by_side(left, right)

In [ ]:
# inner join
left, right = d1.align(d2, join='inner')
render_df_side_by_side(d1,d2)

In [ ]:
# we can fill the NaNs with a certain value if needed
# let's fill with -100 so we can see it clearly
left, right = d1.align(d2, join='outer', fill_value = -100) 
render_df_side_by_side(left, right)

In [ ]:
# left join: left index is added to right, but not the other way
left, right = d1.align(d2, join='left') 
render_df_side_by_side(left, right)

In [ ]:
# right join: right index is added to left, but not the other way
left, right = d1.align(d2, join='right') 
render_df_side_by_side(left, right)

There's more to explore in align, and it can get confusing, so beginners be careful, take time and try to work out the result before you execute to build intuition.

# Operations on data

## Stats

Operations in general exclude missing data

In [ ]:
# arithmetic mean, for each column (axis = 0)
df.mean()

In [ ]:
# mean across a row (axis = 1)
df.mean(1)